In [59]:
# !pip install imbalanced-learn
# !pip install sdv
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sdv.tabular import GaussianCopula
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
import os
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier
#regularization


In [63]:
def load_data(load_train =True):
    if load_train:
        has_fraudulent = False
    else:
        has_fraudulent=True
    first_file = True
    if load_train:
        directory = 'train'
    else:
        directory = 'test'
    for filename in os.listdir(directory):
        if first_file:
            df = pd.read_csv(os.path.join(directory, filename),index_col=0)
            first_file = False
            if df.columns.__contains__('fraudulent'):
                has_fraudulent = True
        else:
            df2 = pd.read_csv(os.path.join(directory, filename),index_col=0)
            if df2.columns.__contains__('fraudulent') :
                if has_fraudulent==True:
                    df2.pop('fraudulent')
                else:
                    has_fraudulent=True
            df = pd.merge(df, df2, left_index=True, right_index=True)
    col = df.columns
    col = list(col)
    col.sort()
    df = df[col]
    print(df.shape)
    return df


In [64]:
X = load_data()
y = X.pop('fraudulent')

(10728, 20040)


In [69]:
import warnings
warnings.filterwarnings('ignore')

models = [
    # KNeighborsClassifier(n_neighbors=1),
    # KNeighborsClassifier(n_neighbors=3),
    # KNeighborsClassifier(n_neighbors=5),
    # svm.SVC(max_iter=99999, C=1000),
    # svm.SVC(max_iter=99999, C=5),
    # LogisticRegression(random_state=0)
    # RandomForestClassifier(max_depth=100, random_state=0),
    RandomForestClassifier(max_depth=1000, random_state=0),
    GradientBoostingClassifier(n_estimators=1000, learning_rate=0.1, max_depth=5, random_state=0),
    GradientBoostingClassifier(n_estimators=1000, learning_rate=0.1, random_state=0),
    # GradientBoostingClassifier(n_estimators=10000, learning_rate=0.1, max_depth=5, n_iter_no_change=2000, random_state=0),
    # MLPClassifier(hidden_layer_sizes=(1000,1000,100,60))
]
skf = StratifiedKFold(n_splits=2)
best_model = None
best_score = 0
for model in models:
    y_pred_all, y_test_all = pd.DataFrame(),pd.DataFrame()
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]


        ##feature selection
        clf = ExtraTreesClassifier(n_estimators=1000)
        clf = clf.fit(X_train, y_train)
        selector = SelectFromModel(estimator=clf,prefit=True,max_features=200)
        X_train = selector.transform(X_train)
        X_test=selector.transform(X_test)
        X_train,X_test = pd.DataFrame(X_train), pd.DataFrame(X_test)
        integers = X_train.columns
        stringed = map(str,integers)
        X_train.columns = stringed


        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        y_pred_all = pd.concat([y_pred_all,pd.DataFrame(y_pred)],ignore_index=True)
        y_test_all =pd.concat( [y_test_all,pd.DataFrame(y_test)],ignore_index=True)
    print(model.__class__)
    score = f1_score(y_test_all, y_pred_all)
    if score > best_score:
        best_score = score
        best_model = model
    print(score)



<class 'sklearn.ensemble._forest.RandomForestClassifier'>
0.7734939759036144
<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>
0.7803790412486066
<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>
0.7813540510543839


In [ ]:
print('best model:',best_model.__class__)
print('With score:', best_score)
print('150 features:GradientBoostingClassifier(n_estimators=1000, learning_rate=0.1, random_state=0) ')

In [39]:
X

,Associate_Degree_education,Associate_experience,Bachelor's_Degree_education,Certification_education,College_Coursework_education,Contract_employment,Director_experience,Doctorate_education,Entry_level_experience,Executive_experience,...,title_yorks,title_yorkshire,title_young,title_youth,title_yr,title_yrs,title_zambia,title_zealand,title_zend,title_zoekt
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10723,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10724,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10725,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10726,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
test = load_data(load_train=False)

clf = ExtraTreesClassifier(n_estimators=200)
clf = clf.fit(X, y)
selector = SelectFromModel(estimator=clf,prefit=True,max_features=100)
X = selector.transform(X)
test=selector.transform(test)
X,test = pd.DataFrame(X), pd.DataFrame(test)
integers = X.columns
stringed = map(str,integers)
X.columns = stringed

best_model.fit(X,y)
res = best_model.predict(test)
res = pd.DataFrame(res)
res.index.name = 'Id'
res.rename(columns={0: 'Category'}, inplace=True, errors='raise')
res.to_csv('Kaggle_file_to_submit/prediction_tokenisation.csv')
res

(7152, 10047)


,Category
Id,
0,0
1,1
2,0
3,0
4,0
...,...
7147,0
7148,0
7149,0


In [165]:
# if there is an error in the above code with differences it means someone uploaded different formats for the train and test files. Below are the difference in columns
set(test.columns.to_list()) - set(X.columns.to_list())

set()

In [166]:
set(X.columns.to_list()) - set(test.columns.to_list())

set()

In [167]:
X.shape

(10728, 259)

In [51]:
test=load_data(load_train=False)

(7152, 10039)


In [15]:
sum(y)

494

In [49]:
test.shape

(7152, 100)

In [41]:
type(X_train)

numpy.ndarray

In [50]:
d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data=d)

In [52]:
integers = df.columns
stringed = (map(str,integers))
df.columns = stringed

In [53]:
df

,col1,col2
0,1,3
1,2,4


In [52]:
df = pd.read_csv('test/peter_test7.csv')

In [58]:
df

,Unnamed: 0,title_jr,title_business,title_analyst,title_quality,title_entry,title_level,title_english,title_teacher,title_abroad,...,benefits_rata,benefits_commuting,benefits_needed,benefits_snowboarding,benefits_cooking,benefits_declara,benefits_trends,benefits_covered,benefits_leapfrog,benefits_nac
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7147,7147,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7148,7148,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7149,7149,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7150,7150,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
